In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from mpi4py import MPI
import matplotlib.pyplot as plt
from vpm_py import VPM
from vpm_py.console_io import print_IMPORTANT,print_red
from test_problems.oseen_vortex import oseen_assign
from vpm_py.visualization import StandardVisualizer

In [3]:
def initialize_oseen_vortex(
    vpm: VPM, 
    gamma: float, 
    viscosity: float,
    density: float,
    t: float 
):
    # Create particles
    NVR = 100
    neq = vpm.num_equations

    XPR_zero = np.zeros((3, NVR), dtype=np.float64)
    XPR_zero[:, 0] = [-1 , -1 , -2.]
    XPR_zero[:, 1] = [ 1 ,  1 ,  2.]
    QPR_zero = np.ones((neq + 1, NVR), dtype=np.float64)

    # Initialization VPM
    vpm.vpm_define(
        num_equations=neq,
        particle_positions= XPR_zero, 
        particle_charges= QPR_zero, 
    )
    
    # Initialize Hill Vortex
    st = MPI.Wtime()
    print_IMPORTANT("Hill vortex initialization")
    velocity_hill, vorticity_hill, pressure_hill = oseen_assign(
        vpm = vpm,
        viscosity= viscosity,
        density= density,
        t = t, 
        gamma= gamma
    )
    vpm.particle_mesh.set_rhs_pm(vorticity_hill)
    print_red("Setting RHS_pm as computed from the hill vortex")
    
    st = MPI.Wtime()
    print_red("Remeshing")
    XPR_hill, QPR_hill = vpm.remesh_particles(project_particles=False, cut_off=-1.) 
    et = MPI.Wtime()
    print(f"\tRemeshing finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")

    print_IMPORTANT("Particles initialized")
    NVR = vpm.particles.NVR
    return XPR_hill, QPR_hill, NVR, velocity_hill, vorticity_hill, pressure_hill

In [4]:
REYNOLDS_NUMBER=10
REMESH_FREQUENCY=40
apply_vorticity_correction=False

In [5]:
# PROBLEM STATEMENT
UINF = np.array([0.0, 0.0, 1.0])
gamma = 2.0
# Reynolds number = U * L / nu , where U is the velocity, L is the radius of the sphere and nu is the kinematic viscosity
# nu = U * L / REYNOLDS_NUMBER
VISCOSITY = 10e-3 
DENSITY = 1000.
# DT should be set according to the CFL condition: CFL = U * DT / dx < 1
# dpm = np.array([0.1, 0.1, 0.1])
dpm = np.array([0.02, 0.02, 0.02])
# dpm = np.array([0.04, 0.04, 0.04])

# CASE FOLDER
CASE_FOLDER = "/mnt/c/Users/tryfonas/Data/hill_vortex_5_seconds/oseen_pressure"

In [6]:
# Initialize MPI
comm = MPI.COMM_WORLD
start_time = MPI.Wtime()
rank = comm.Get_rank()
np_procs = comm.Get_size()
neq = 3
# Initialize VPM
vpm = VPM(
    number_of_equations=3,
    number_of_processors=np_procs,
    rank=rank,
    verbocity=0,
    dx_particle_mesh=dpm[0],
    dy_particle_mesh=dpm[1],
    dz_particle_mesh=dpm[2],
    case_folder=CASE_FOLDER,
)
plotter = StandardVisualizer(
    plot_particles=("charge", "magnitude"),  # plot_mesh=("velocity", "magnitude")
)
vpm.attach_visualizer(plotter)

Number of processors: 1
NBI: 1, NBJ: 1, NBK: 1
------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                Initializing the plot                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
Number of plots: 1
Rows: 1, Columns: 1


In [7]:
(
    XPR_zero, QPR_zero, NVR, velocity_oseen, vorticity_oseen, pressure_oseen
) = initialize_oseen_vortex(
    vpm= vpm,
    gamma= gamma,
    viscosity= VISCOSITY,
    density= DENSITY,
    t=0.02
)
# Create the plot to live update the particles
vpm.update_plot(
    f"Time: {0:.2f}s",
)

------------------------------------------------------------------------------------------------------------------------------------------------------
                                                              Hill vortex initialization                                                              
------------------------------------------------------------------------------------------------------------------------------------------------------
Getting analytical solution
CP shape: (113, 113, 209, 3)
Calculating Oseen vortex with gamma: 2.0, viscosity: 0.01, density: 1000.0, t: 0.02


/mnt/e/VPM/test/test_problems/oseen_vortex.py:28: RuntimeWarning: divide by zero encountered in divide
  u_theta = (D/r) *(1 - np.exp(exponent))
/mnt/e/VPM/test/test_problems/oseen_vortex.py:28: RuntimeWarning: invalid value encountered in multiply
  u_theta = (D/r) *(1 - np.exp(exponent))
/mnt/e/VPM/test/test_problems/oseen_vortex.py:31: RuntimeWarning: divide by zero encountered in divide
  -1/(2*r**2) +


Analytical vorticity: (113, 113, 209, 3)
Setting RHS_pm as computed from the hill vortex
Remeshing
Applying Remeshing with Energy Cutoff Value of 99.99%
	Remeshing finished in 0m 0.50s

------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                Particles initialized                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
	Particle plots updated in 0.02s
	Title updated in 0.00s
	Plot drawn in 0.00s
	Events flushed in 0.34s


In [8]:
vpm.vpm_solve_velocity_deformation(
    timestep=0,
    num_equations=neq,
    particle_positions    =  XPR_zero,
    particle_charges      =  QPR_zero,
)
vpm.vpm_solve_pressure(density= DENSITY, viscosity= VISCOSITY)

 
Solving Poisson problem for the vorticity field
 
 
 Total enstrophy particles   2.0885240581072465E-007
 Total vorticity particles   6.5414551837643717E-005
 Total momentum x particles   9.1711496006412305E-004
 Total momentum y particles  -1.0732067862772358E-003
 Total momentum z particles   0.0000000000000000     
 
Solving Poisson for the Pressure field. ρ =  1000.00 ν =     0.01
 The divergence of the stress tensor
 	Max divergence of the stress tensor   196.41471181120954     
 	Min divergence of the stress tensor  -196.41661859800394     
 	Total divergence of the stress tensor   1.7595469724426493E-002
 The divergence of the cross product
 	Max divergence of the cross product   58889.072537060318     
 	Min divergence of the cross product  -275278.57842730184     
 	Total divergence of the cross product  -7.2615937276646863E-010
 


In [9]:
U_PM = vpm.particle_mesh.velocity.copy()
# Convert U_PM to c order
U_PM = np.ascontiguousarray(U_PM, dtype=np.float64)

U_MESH = np.nan_to_num(U_PM, posinf=0.0, neginf=0.0)[0, :, :, :]
U_OSEEN = np.nan_to_num(velocity_oseen, posinf=0.0, neginf=0.0)[0, :, :, :]

# Get the error between the computed pressure and the hill vortex pressure
error = np.abs(U_MESH - U_OSEEN) 
print(f"Error: {np.max(error)}")
print(f"Error: {np.mean(error)}")


XYZ = vpm.particle_mesh.grid_positions

z_shape = XYZ.shape[3]
z_shapes = [z_shape // 2]
for idx_z in z_shapes:
    # Get the actual z value
    z = XYZ[2, 0, 0, idx_z]
    # Assert that the z value is the same for all the grid
    assert np.allclose(XYZ[2, :, :,idx_z], z)
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Z = {z}")

    plot = ax[0].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1, :, :, idx_z], U_MESH[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[0])
    ax[0].set_title("Computed pressure (VPM)")

    plot = ax[1].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1, :, :, idx_z], U_OSEEN[:, :, idx_z], cmap = 'turbo') 
    fig.colorbar(plot, ax=ax[1])
    ax[1].set_title("Analytical pressure")

    plot = ax[2].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1,:, :, idx_z], error[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[2])
    ax[2].set_title("Error")
    plt.show()

Error: 6.720512885098528
Error: 0.049891419778812264


In [10]:
P_MESH = np.nan_to_num(vpm.particle_mesh.pressure, posinf=0.0, neginf=0.0)
P_OSEEN = np.nan_to_num(pressure_oseen, posinf=0.0, neginf=0.0)

# Get the error between the computed pressure and the hill vortex pressure
error = np.abs(P_MESH - P_OSEEN) 
print(f"Error: {np.max(error)}")
print(f"Error: {np.mean(error)}")

XYZ = vpm.particle_mesh.grid_positions
z_shape = XYZ.shape[3]
z_shapes = [z_shape // 2]
for idx_z in z_shapes:
    # Get the actual z value
    z = XYZ[2, 0, 0, idx_z]
    # Assert that the z value is the same for all the grid
    assert np.allclose(XYZ[2, :, :,idx_z], z)
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Z = {z}")

    plot = ax[0].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1, :, :, idx_z], P_MESH[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[0])
    ax[0].set_title("Computed pressure (VPM)")

    plot = ax[1].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1, :, :, idx_z], P_OSEEN[:, :, idx_z], cmap = 'turbo') 
    fig.colorbar(plot, ax=ax[1])
    ax[1].set_title("Analytical pressure")

    plot = ax[2].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1,:, :, idx_z], error[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[2])
    ax[2].set_title("Error")
    plt.show()

Error: 96423.897954211
Error: 69.25904983671592


In [11]:
U_PM

array([[[[-0.06405374, -0.06490997, -0.06591728, ..., -0.06591795,
          -0.0649127 , -0.06405442],
         [-0.06424288, -0.06508059, -0.06596268, ..., -0.06596337,
          -0.06508128, -0.06424357],
         [-0.06427679, -0.06514425, -0.06594673, ..., -0.06594742,
          -0.0651447 , -0.06427748],
         ...,
         [ 0.06427661,  0.06514408,  0.06594655, ...,  0.06594741,
           0.06514469,  0.06427746],
         [ 0.06424271,  0.06508043,  0.06596253, ...,  0.06596337,
           0.06508128,  0.06424357],
         [ 0.06405359,  0.06490979,  0.06591714, ...,  0.06591797,
           0.06491164,  0.06405443]],

        [[-0.06552457, -0.06634793, -0.06715471, ..., -0.06715556,
          -0.06634782, -0.06552528],
         [-0.06547149, -0.06637939, -0.067235  , ..., -0.06723572,
          -0.06637963, -0.06547219],
         [-0.06544539, -0.066415  , -0.06729846, ..., -0.06729913,
          -0.06641557, -0.06544609],
         ...,
         [ 0.0654452 ,  0.06641483